In [1]:
# Import libraries

import os
import requests
from io import StringIO

import psycopg2
from psycopg2 import sql

# Data Manipulation
import pandas as pd
import geopandas as gpd

# Getting .env information (API keys, etc.)
from dotenv import load_dotenv

In [3]:
# Get REDCap token

load_dotenv()

redCap_token_signUp = os.getenv('REDCAP_TOKEN_SIGNUP') # Survey Token
redCap_token_report = os.getenv('REDCAP_TOKEN_REPORT') # Report Token

# Get Database credentials

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))    

In [4]:
# get the highest record id we've got

# Connect
conn = psycopg2.connect(**pg_connection_dict) 
# Create cursor
cur = conn.cursor()

cmd = sql.SQL('''SELECT MAX(record_id)
FROM "Sign Up Information";;
''')

cur.execute(cmd) # Execute
conn.commit() # Committ command

# Unpack response into pandas series

max_record_id = cur.fetchall()[0][0]

# Close cursor
cur.close()
# Close connection
conn.close()

In [5]:
# filterLogic Redcap API
# Redcap logic guide - https://cctsi.cuanschutz.edu/docs/librariesprovider28/redcap/redcap-logic-guide.pdf?sfvrsn=258e94ba_2

### Select by record_id

# # They must be strings before below

# record_ids = ['0', '1', '2', '3']
# filterLogic_record_id = '[record_id]=' + ' OR [record_id]='.join(record_ids)

# ### Select by report (to the City)
# filterLogic_report = '[report]=1'

### Select all
filterLogic_str = f"[record_id]>{max_record_id}"

In [6]:
data = {
    'token': redCap_token_signUp,
    'content': 'record',
    'action': 'export',
    'format': 'csv',
    'type': 'flat',
    'csvDelimiter': '',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'csv',
    'filterLogic': filterLogic_str  
}
r = requests.post('https://redcap.ahc.umn.edu/api/',data=data)

In [ ]:
print('HTTP Status: ' + str(r.status_code))
df = pd.read_csv(StringIO(r.text))



In [12]:
r.status_code == 200 and r.text != '\n'

False

In [28]:
#r.text

In [29]:
# Spatialize dataframe

gdf = gpd.GeoDataFrame(df, 
                        geometry = gpd.points_from_xy(
                                    df.lon,
                                    df.lat,
                                    crs = 'EPSG:4326')
                               )

gdf['wkt'] = gdf.geometry.apply(lambda x: x.wkt)

In [30]:
# Upload to database

# Prep

focus_df = gdf[['record_id', 'wkt']]
cols_for_db = ['record_id', 'geometry']

In [31]:
# Connect to PostGIS Database

conn = psycopg2.connect(**pg_connection_dict)

In [32]:
#connect to the cursor
cur = conn.cursor()

### iterate over the dataframe and insert each row into the database using a SQL INSERT statement

for index, row in final_df.iterrows():

    q1 = sql.SQL('INSERT INTO "Sign Up Information" ({}) VALUES ({},{});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db)-1)),
     sql.SQL('ST_SetSRID(ST_GeomFromText(%s), 4326)::geometry'))
    # print(q1.as_string(conn))
    # print(row)
    # break
    
    cur.execute(q1.as_string(conn),
        (list(row.values))
        )
    # Commit command

    conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

In [24]:
message = '''Welcome to SpikeAlerts! 

We will text 8am-9pm when air quality seems unhealthy within 1 kilometer of your designated location.

If you have questions, please email mplsairquality@gmail.com or see https://SpikeAlerts.github.io/Website

Reply STOP to end this service'''

In [25]:
print(message)
print('\n\n MESSAGE LENGTH:', len(message), 'Characters') # TWILIO charges by segment (160 characters)

Welcome to SpikeAlerts! 

We will text 8am-9pm when air quality seems unhealthy within 1 kilometer of your designated location.

If you have questions, please email mplsairquality@gmail.com or see https://SpikeAlerts.github.io/Website

Reply STOP to end this service


 MESSAGE LENGTH: 266 Characters
